# 火币基础数据收集（Huobi Historic Data Collection）

**When Using a Proxy：**

File: \Huobi\Impl\websocketconnection.py 

connection_instance.ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE}, http_proxy_host="127.0.0.1", http_proxy_port=1881)

In [ ]:
from huobi import *
from huobi.model import *

import json
import time
import pandas as pd
import datetime

In [ ]:
import mysql.connector as db

db_config = {
    "database": "Huobi_Ex",
    "user": "root",
    "password": "root",
    "host": "localhost",
    "port": 3306,
}

# 连接数据库获取游标，可以设置返回数据的格式，元组，命令元组，字典等...
connect = db.Connect(**db_config)
cursor = connect.cursor()

In [ ]:
subscription_client = SubscriptionClient(api_key="ae957d1d-e1d269ee-18b8966a-bewr5drtmh", secret_key="ca77afb2-b0eb07e2-ad910eae-6df30")

#subscription_client = SubscriptionClient()
subscription_client.unsubscribe_all()

def callback(candlestick_event: 'CandlestickEvent'):
    sqli = "replace into `kline_%(symbol)s_%(period)s` (`amount`, `open`, `close`, `high`, `id`, `count`, `low`, `vol`) \
        select %(amount)s, %(open)s, %(close)s, %(high)s, %(id)s, %(count)s, %(low)s, %(vol)s where \
        not exists(select id from `kline_%(symbol)s_%(period)s` where id = %(id)s and amount = %(amount)s) "
    

    
    #print(candlestick_event.symbol)
    #print(candlestick_event.interval)
    item = {}
    i = 0
    j = 0
    
    for candlestick in candlestick_event.data:
        item.update({"symbol":candlestick_event.symbol,\
                     "period":candlestick_event.interval,\
                     "id":candlestick.id,\
                     "open":candlestick.open,\
                     "close":candlestick.close,\
                     "low":candlestick.low,\
                     "high":candlestick.high,\
                     "amount":candlestick.amount,\
                     "count":candlestick.count,\
                     "vol":candlestick.volume,\
                    })
        sqlcmd = sqli%(item)
        #print(sqlcmd)
        cursor.execute(sqlcmd)
    
        if cursor.rowcount == 2 :
            j += 1
        else :
            i+=cursor.rowcount
        
        #print(cursor.rowcount)
        i += j//2
    
    connect.commit()
    print("keypair:%s :: period:%s,  rows:%s, effect rows:%s, update rows:%s || %s"%(candlestick_event.symbol.rjust(10,' '), candlestick_event.interval.rjust(8,' '), str(len(candlestick_event.data)).rjust(5,' '), str(i).rjust(5,' '), str(j).rjust(5,' '),  item["id"]))


In [ ]:
k = 0
n = 3

endtimeflag = datetime.date(2020, 5, 3)
print(int(time.mktime(endtimeflag.timetuple())))

In [ ]:
#symbol: 'btcusdt', 'ethusdt', 'htusdt', 'xrpusdt', 'ltcusdt', 'bchusdt', 'eosusdt', 'etcusdt', \
#         'bsvusdt', 'trxusdt', 'xmrusdt', 'hb10usdt', 'zecusdt', 'dashusdt', 'adausdt', 'iotausdt', \
#         'newusdt', 'seeleusdt', 'wiccusdt', 'ontusdt', 'mxusdt', 'hptusdt', 'ektusdt', 'gtusdt', \
#         'lambusdt', 'bixusdt', 'linkusdt', 'algousdt', 'wavesusdt', 'vidyusdt', 'atomusdt', 'iostusdt', 'qtumusdt'

#interval: 1min, 5min, 15min, 30min, 60min, 4hour, 1day, 1mon, 1week, 1year
#from : [1501174800, 2556115200]
#to :[1501174800, 2556115200]
#1582500000 : Timestamp('2020-02-24 07:20:00')
#size : (0,300]

#主流前9个

periods = ['1min', '5min', '15min', '30min', '60min', '4hour']

# symbol =  ['btcusdt', 'ethusdt', 'xrpusdt', 'ltcusdt', 'bchusdt', 'eosusdt', 'etcusdt', \
#          'bsvusdt', 'trxusdt', 'xmrusdt', 'hb10usdt', 'linkusdt', 'zecusdt', 'dashusdt', 'adausdt', 'iotausdt', \
#          'newusdt', 'seeleusdt', 'wiccusdt', 'ontusdt', 'mxusdt', 'hptusdt', 'ektusdt', 'gtusdt', \
#          'lambusdt', 'bixusdt', 'algousdt', 'wavesusdt', 'vidyusdt', 'atomusdt', 'iostusdt', 'qtumusdt']

#symbol =  ['htbtc', 'ethbtc', 'etcbtc', 'bsvbtc', 'linkbtc', 'xrpbtc', 'ltcbtc', 'bchbtc', 'eosbtc']
#symbol =  ['htusdt', 'ethusdt', 'etcusdt', 'bsvusdt', 'linkusdt', 'xrpusdt', 'ltcbtc', 'bchusdt', 'eosusdt']

symbol = ['btcusdt', 'ethusdt', 'htusdt', 'xrpusdt', 'ltcusdt', 'bchusdt', 'eosusdt', 'etcusdt', 'bsvusdt', 'linkusdt', \
          'ethbtc', 'htbtc', 'xrpbtc', 'ltcbtc', 'bchbtc', 'eosbtc', 'etcbtc', 'bsvbtc', 'linkbtc', ]

timestep = [60, 300, 900, 1800, 3600, 14400] 
step = 300
fromtime = 1517464800 # htbtc
fromtime = 1508990400 + 0 * step * timestep[k] if(n==1 or n==10)  else fromtime  #ethbtc
fromtime = 1509422460 + 0 * step * timestep[k] if(n==2 or n==11) else fromtime  #etcbtc
fromtime = 1543564800 + 0 * step * timestep[k] if(n==3 or n==12) else fromtime  #bsvbtc
fromtime = 1517464800 + 0 * step * timestep[k] if(n==4 or n==13) else fromtime  #linkbtc
fromtime = 1511452800 + 0 * step * timestep[k] if(n==5 or n==14) else fromtime  #xrpbtc
fromtime = 1501237440 + 0 * step * timestep[k] if(n==6 or n==15) else fromtime  #ltcbtc
fromtime = 1509292800 + 0 * step * timestep[k] if(n==7 or n==16) else fromtime  #bchbtc
fromtime = 1510934400 + 0 * step * timestep[k] if(n==8 or n==17) else fromtime  #eosbtc
fromtime = 1545127200 + 0 * step * timestep[k] if(n==9 or n==18) else fromtime  #linkbtc

#fromtime = 1588348800 #step in point

sfromtime = fromtime

endtime = 1588435200 #end point

In [ ]:
print(len(symbol))

In [ ]:
while fromtime < endtime and n < len(symbol) :
    subscription_client.request_candlestick_event(symbol[n], periods[k], callback, from_ts_second=fromtime,end_ts_second=fromtime + step * timestep[k], auto_close=True, error_handler=None) #1582500000
    time.sleep(5)
    fromtime = fromtime + step * timestep[k]

    if fromtime >= endtime :
        k = k+1
        fromtime = sfromtime
        
    if k == 6 :
        k = 0
        n = n+1
        fromtime = sfromtime

time.sleep(60)
cursor.close()
